## Import

In [ ]:
import pandas as pd

## Read data

In [ ]:
data = pd.read_csv('your data path', sep="\t")

## Some Data Processing

In [ ]:
data = data.drop(columns=['C', 'W', 'D', 'P', 'Unnamed: 12'])
data.loc[data['GC'] == 'Non-GC', 'GC'] = 0
data.loc[data['GC'] == 'GC', 'GC'] = 1
data = data[data['Timetick'] != 0] 

## See some data status

In [ ]:
# timetick dtype are int64
print( 'timetick max : ', data['Timetick'].max())
print( 'timetick min : ', data['Timetick'].min())
print( 'victim_block_invalid_page max : ', data['victim_block_invalid_page'].max())
print( 'victim_block_invalid_page min : ', data['victim_block_invalid_page'].min())
print( 'victim_block_invalid_page mean : ', data['victim_block_invalid_page'].mean())
print( 'victim_block_invalid_page quantile : ', data['victim_block_invalid_page'].quantile(.75))
print( 'victim_block_invalid_page median : ', data['victim_block_invalid_page'].median())
print( 'queue size : ', data['queue_size'].max())
print( 'free block min: ', data['freeblocks'].min())
# freeblock max 2048

## Timetick and Free Blocks Normorlization

In [ ]:
data['Timetick'] = (data['Timetick'] - data['Timetick'].min()) / (data['Timetick'].max() - data['Timetick'].min())
data['freeblocks'] = data['freeblocks'] / 2048

## Seperate GC and NonGC

In [ ]:
gc = data[data['GC'] == 1]
ngc = data[data['GC'] == 0]

## GC Transaction Queue and the invalid pages of victim block Normalization

In [ ]:
data['queue_size'] = (data['queue_size'] - data['queue_size'].min()) / (data['queue_size'].max() - data['queue_size'].min())
data['victim_block_invalid_page'] = (data['victim_block_invalid_page'] - data['victim_block_invalid_page'].min()) / (data['victim_block_invalid_page'].max() - data['victim_block_invalid_page'].min())

## if __queue_size>0__ and __GC = 1__  random choose some data __GC = 1 -> GC = 0__ 
#### the queue size > 0 and keep doing GC will increase the GC overhead( I/O traffic )


In [ ]:
noise = data[data['queue_size'] > 0]
noise.shape
noise = noise[noise['GC'] == 1]
print('queue _size > 0 ', noise.shape)
qGCtoNonGC = noise.sample( n="depend your data", random_state=1)
qGCtoNonGC['GC'] = 0
print('randomly select ', qGCtoNonGC.shape)

## drop the data you random select

In [ ]:
noise_q = noise.drop(qGCtoNonGC.index)

In [ ]:
print(noise_q.shape)
print(noise.shape)

## Also drop the noise from origin data

In [ ]:
print(data.shape)
data = data.drop(noise.index)
print(data.shape)

## If the invalid pages of victim block < third quatile
### We need label the data from GC to NonGC( Avoid increasing GC overhead )

In [ ]:
# victim block invalid page < third quatile GC->NonGC
noise_victim = data[data['victim_block_invalid_page'] < data['victim_block_invalid_page'].quantile(.75)]
# print(noise_victim.shape)
noise_victim = noise_victim[noise_victim['GC'] == 1]
print(noise_victim.shape)

#### vGCtoNonGC -> a list of invalid pages of victim block from GC to NonGC

In [ ]:
vGCtoNonGC = noise_victim.sample( n="depend your data", random_state=1)
vGCtoNonGC['GC'] = 0
print(vGCtoNonGC.shape)

In [ ]:
noise_victimGC = noise_victim.drop(vGCtoNonGC.index)
print(noise_victim.shape)
print(noise_victimGC.shape)

In [ ]:
print(data.shape)
data = data.drop(noise_victim.index)
print(data.shape)

In [ ]:
GC = data[data['GC'] == 1]
nonGC = data[data['GC'] == 0]
print(GC.shape)
print(nonGC.shape)

### Merge every list of noise data

In [ ]:
noise_GC = noise_q.append(noise_victimGC, ignore_index=True)
noise_nonGC = qGCtoNonGC.append(vGCtoNonGC, ignore_index=True)
print(noise_GC.shape)
print(noise_nonGC.shape)

### Random select the original data GC and NonGC
### Merge the noise data to be a training data

In [ ]:
GC_total = GC.sample( n="depend your data", random_state=1 )
data = data.drop(GC_total.index)
GC_total = GC_total.append(noise_GC, ignore_index=True)
print(GC_total.shape)

In [ ]:
nonGC_total = nonGC.sample( n="depend your data", random_state=1 )
data = data.drop(nonGC_total.index)
nonGC_total = nonGC_total.append(noise_nonGC, ignore_index=True)
print(nonGC_total.shape)

In [ ]:
print(data.shape)

In [ ]:
training = GC_total.append(nonGC_total, ignore_index=True)

In [ ]:
training.to_csv('training_data.csv', sep='\t', encoding='utf-8', index=False)